In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/954471/954471/Train/M/121368482_3314461121964215_543249990768154260_o.jpg
/kaggle/input/954471/954471/Train/M/121338993_3559428640790674_3898861408853482856_o.jpg
/kaggle/input/954471/954471/Train/M/121543339_3481304788558895_349846518547631903_n.jpg
/kaggle/input/954471/954471/Train/M/121335583_2114014788729050_910466790392877863_n.jpg
/kaggle/input/954471/954471/Train/M/121478814_3277574072359599_3175620794088564281_n.jpg
/kaggle/input/954471/954471/Train/M/121381660_3348014425282334_9154920501988623614_o.jpg
/kaggle/input/954471/954471/Train/M/121472960_3362092607235628_5170860563284396140_o.jpg
/kaggle/input/954471/954471/Train/M/121222913_2888173951406016_2943121663743613328_n.jpg
/kaggle/input/954471/954471/Train/F/121256709_972711336544125_7929623606048122247_o.jpg
/kaggle/input/954471/954471/Train/F/121597221_972711616544097_204099069773999800_o.jpg
/kaggle/input/954471/954471/Train/F/121282643_1739796282838922_3924762867765759060_n.jpg
/kaggle/input/954471/954471

# Load Image 

In [2]:
test_dir="/kaggle/input/954471/954471/Test"
train_dir="/kaggle/input/954471/954471/Train"

train_dir_f = train_dir + '/F'
train_dir_m = train_dir + '/M'
test_dir_f = test_dir + '/F'
test_dir_m = test_dir + '/M'

In [3]:
print('number of f training images - ',len(os.listdir(train_dir_f)))
print('number of m training images - ',len(os.listdir(train_dir_m)))
print('number of f testing images - ',len(os.listdir(test_dir_f)))
print('number of m testing images - ',len(os.listdir(test_dir_m)))

number of f training images -  8
number of m training images -  8
number of f testing images -  4
number of m testing images -  4


ImageDataGenerator

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        rescale = 1.0/255.0,   # Intensity Normalized
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,   # randomly flip images
        validation_split=0.2)

In [5]:
batch_size = 2
img_height = 256 
img_width  = 256
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = data_generator.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data


Found 14 images belonging to 2 classes.
Found 2 images belonging to 2 classes.


In [6]:
train_generator.class_indices

{'F': 0, 'M': 1}

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
# preparing the layers in the Convolutional Deep Neural Network
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = train_generator.image_shape))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.3))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Conv2D(filters = 126, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.15))
model.add(Flatten())
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dropout(rate = 0.3))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(rate = 0.15))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [8]:
fitted_model = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 250)

Epoch 1/250
7/7 [==============================] - 1s 200ms/step - loss: 0.9527 - accuracy: 0.5000 - val_loss: 0.6916 - val_accuracy: 0.5000
Epoch 2/250
7/7 [==============================] - 1s 146ms/step - loss: 0.7158 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/250
7/7 [==============================] - 1s 171ms/step - loss: 0.6944 - accuracy: 0.4286 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 4/250
7/7 [==============================] - 1s 210ms/step - loss: 0.6938 - accuracy: 0.3571 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/250
7/7 [==============================] - 1s 148ms/step - loss: 0.6935 - accuracy: 0.4286 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 6/250
7/7 [==============================] - 1s 142ms/step - loss: 0.6947 - accuracy: 0.5714 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/250
7/7 [==============================] - 1s 154ms/step - loss: 0.6934 - accuracy: 0.4286 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 8/250
7

7/7 [==============================] - 1s 147ms/step - loss: 0.0548 - accuracy: 0.9286 - val_loss: 9.3720e-05 - val_accuracy: 1.0000
Epoch 59/250
7/7 [==============================] - 1s 153ms/step - loss: 0.0593 - accuracy: 0.9286 - val_loss: 4.2982e-05 - val_accuracy: 1.0000
Epoch 60/250
7/7 [==============================] - 1s 142ms/step - loss: 0.0216 - accuracy: 1.0000 - val_loss: 6.6809e-05 - val_accuracy: 1.0000
Epoch 61/250
7/7 [==============================] - 1s 153ms/step - loss: 0.1896 - accuracy: 0.8571 - val_loss: 2.0429e-06 - val_accuracy: 1.0000
Epoch 62/250
7/7 [==============================] - 1s 144ms/step - loss: 0.1335 - accuracy: 0.8571 - val_loss: 2.5585e-04 - val_accuracy: 1.0000
Epoch 63/250
7/7 [==============================] - 1s 146ms/step - loss: 0.2485 - accuracy: 0.8571 - val_loss: 9.6199e-04 - val_accuracy: 1.0000
Epoch 64/250
7/7 [==============================] - 1s 146ms/step - loss: 0.0732 - accuracy: 0.9286 - val_loss: 3.2134e-04 - val_accuracy

Epoch 114/250
7/7 [==============================] - 1s 142ms/step - loss: 3.0316e-04 - accuracy: 1.0000 - val_loss: 4.0953e-05 - val_accuracy: 1.0000
Epoch 115/250
7/7 [==============================] - 1s 146ms/step - loss: 0.0483 - accuracy: 1.0000 - val_loss: 2.0421e-06 - val_accuracy: 1.0000
Epoch 116/250
7/7 [==============================] - 1s 144ms/step - loss: 0.1440 - accuracy: 0.9286 - val_loss: 5.3216e-04 - val_accuracy: 1.0000
Epoch 117/250
7/7 [==============================] - 1s 144ms/step - loss: 0.0246 - accuracy: 1.0000 - val_loss: 2.6725e-06 - val_accuracy: 1.0000
Epoch 118/250
7/7 [==============================] - 1s 140ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 4.2470e-07 - val_accuracy: 1.0000
Epoch 119/250
7/7 [==============================] - 1s 142ms/step - loss: 0.0450 - accuracy: 1.0000 - val_loss: 2.4829e-07 - val_accuracy: 1.0000
Epoch 120/250
7/7 [==============================] - 1s 143ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 1

Epoch 170/250
7/7 [==============================] - 1s 143ms/step - loss: 0.9151 - accuracy: 0.9286 - val_loss: 2.1096e-05 - val_accuracy: 1.0000
Epoch 171/250
7/7 [==============================] - 1s 143ms/step - loss: 0.4169 - accuracy: 0.9286 - val_loss: 0.2467 - val_accuracy: 1.0000
Epoch 172/250
7/7 [==============================] - 1s 143ms/step - loss: 2.0438 - accuracy: 0.7143 - val_loss: 3.3551e-14 - val_accuracy: 1.0000
Epoch 173/250
7/7 [==============================] - 1s 144ms/step - loss: 0.0896 - accuracy: 0.9286 - val_loss: 4.8678e-04 - val_accuracy: 1.0000
Epoch 174/250
7/7 [==============================] - 1s 144ms/step - loss: 0.4263 - accuracy: 0.8571 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 175/250
7/7 [==============================] - 1s 145ms/step - loss: 0.2172 - accuracy: 0.8571 - val_loss: 3.8560e-04 - val_accuracy: 1.0000
Epoch 176/250
7/7 [==============================] - 1s 141ms/step - loss: 0.0557 - accuracy: 1.0000 - val_loss: 1.5663e-06 - 

7/7 [==============================] - 1s 142ms/step - loss: 2.6918e-07 - accuracy: 1.0000 - val_loss: 1.2177e-07 - val_accuracy: 1.0000
Epoch 226/250
7/7 [==============================] - 1s 144ms/step - loss: 2.3753e-05 - accuracy: 1.0000 - val_loss: 3.5286e-08 - val_accuracy: 1.0000
Epoch 227/250
7/7 [==============================] - 1s 141ms/step - loss: 1.6074e-05 - accuracy: 1.0000 - val_loss: 1.1219e-07 - val_accuracy: 1.0000
Epoch 228/250
7/7 [==============================] - 1s 142ms/step - loss: 2.3280e-07 - accuracy: 1.0000 - val_loss: 3.0297e-09 - val_accuracy: 1.0000
Epoch 229/250
7/7 [==============================] - 1s 143ms/step - loss: 0.4013 - accuracy: 0.9286 - val_loss: 2.3536e-06 - val_accuracy: 1.0000
Epoch 230/250
7/7 [==============================] - 1s 142ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 1.0429e-08 - val_accuracy: 1.0000
Epoch 231/250
7/7 [==============================] - 1s 151ms/step - loss: 2.9251e-07 - accuracy: 1.0000 - val_loss:

In [9]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img('../input/954471/954471/Test/F/121558240_972714016543857_6033716185027836421_o.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result[0]))

[1.]


In [10]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img('../input/954471/954471/Test/M/121160480_3678313258869851_5818106878740531607_o.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result[0]))

[1.]


In [11]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img('../input/954471/954471/Test/M/121623008_2688765031372476_1265210388791973144_n.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result[0]))

[1.]
